In [ ]:
import torch
import numpy as np

index_start = -1
index_end = 4
loaded_density_start = np.load(f'../ret/phiflow_{index_start:03d}.npz')["density"]
loaded_density_end = np.load(f'../ret/phiflow_{index_end:03d}.npz')["density"]

device = torch.device("cuda")
raw_density_start = torch.tensor(loaded_density_start, device=device)
raw_density_end = torch.tensor(loaded_density_end, device=device)

In [ ]:
import phi.torch.flow as pf

BOUNDS = pf.Box(x=(0, 1), y=(0, 1), z=(0, 1))
RESOLUTION = (100, 100, 100)

DENSITY = pf.CenteredGrid(
    values=pf.wrap(raw_density_start, pf.spatial('x,y,z')),
    extrapolation=pf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=pf.spatial(x=RESOLUTION[0], y=RESOLUTION[1], z=RESOLUTION[2]),
)
VELOCITY = pf.StaggeredGrid(
    values=0,
    extrapolation=pf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=pf.spatial(x=RESOLUTION[0], y=RESOLUTION[1], z=RESOLUTION[2]),
)

In [ ]:
@pf.jit_compile
def step(v, s, p, dt, bc):
    s = pf.advect.mac_cormack(s, v, dt)
    buoyancy = pf.resample(s * (0, bc, 0), to=v)
    v = pf.advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = pf.fluid.make_incompressible(v, (), pf.Solve('CG', 1e-3, x0=p))
    return v, s, p


dt_train = torch.tensor(0.2, device=device, dtype=torch.float32, requires_grad=True)
bc_train = torch.tensor(0.1, device=device, dtype=torch.float32, requires_grad=True)
optimizer = torch.optim.Adam([dt_train, bc_train], lr=0.1)


def model(v, s, p):
    for i in range(5):
        v, s, p = step(v, s, p, dt_train, bc_train)
    return s.values.native('x,y,z')


num_epochs = 1000
loss_history = []
import tqdm

for epoch in tqdm.trange(num_epochs):
    optimizer.zero_grad()
    ret = model(VELOCITY, DENSITY, None)
    loss = torch.nn.functional.mse_loss(ret, raw_density_end)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    print(f'dt_train: {dt_train}, bc_train: {bc_train}, loss: {loss.item()}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.show()

In [ ]:
import torch
import tqdm

# 生成目标数据（sin(x)的值）
xxx = torch.linspace(-2 * torch.pi, 2 * torch.pi, 100).unsqueeze(1)  # x: 0, 1, ..., 99
target = torch.sin(xxx)  # sin(x) 的目标值


# 定义MLP模型
class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)


# 初始化MLP模型
mlp = MLP(input_dim=1, hidden_dim=64, output_dim=1)

# 定义优化器（Adam）
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.1)

num_epochs = 1000
loss_history = []
decay_rate = 0.1

for epoch in tqdm.trange(num_epochs):
    optimizer.zero_grad()

    mlp_output = mlp(xxx)
    loss = torch.nn.functional.mse_loss(mlp_output, target)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())

    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.1 * (decay_rate ** (epoch / 1000))

In [ ]:
import matplotlib.pyplot as plt

# 绘制优化结果
plt.figure(figsize=(10, 5))
plt.plot(xxx.numpy(), target.numpy(), label='Target (sin(x))', color='blue')
plt.plot(xxx.numpy(), mlp_output.detach().numpy(), label='MLP Output', color='red')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Optimization of sin(x) with MLP using Adam')
plt.show()

# 绘制损失曲线
plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Loss', color='green')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()
plt.show()